# 1. 트랜스포머(Transformer)
- 2017년 구글이 발표한 [논문](https://arxiv.org/abs/1706.03762) 'Attention is all you need' 에서 발표된 모델
- seq2Seq 의 구조인 인코더-디코더를 따르면서도 어텐션(Attention)만으로 구현된 모델
- [트랜스포머](https://wikidocs.net/31379)는 RNN을 사용하지 않고 인코더-디코더 구조를 설계하였음에도 번역 성능에서 RNN보다 우수한 성능을 보여줬으며 2017년 이후 지금까지 다양한 분야에서 사용되는 범용적인 모델

### 1-1. 트랜스포머의 특징
- RNN을 사용하지 않지만 seq2seq 모델의 구조처럼 인코더에서 입력 시퀀스를 입력받고, 디코더에서 출력 시퀀스를 출력하는 인코더-디코더를 사용함
- seq2seq 모델 구조에서는 인코더와 디코더 각각 하나의 RNN 모델처럼 작동했다면, 트랜스포머에서는 인코더와 디코더 단위가 N개를 확장되는 구조(논문에서는 6개씩 사용)

### 1-2. 포지셔널 인코딩(Positional Encoding)
- 트랜스포머는 단어의 위치 정보를 얻기 위해 각 단어의 임베딩 벡터에 위치 정보들을 더하여 모델의 입력으로 사용
- 위치 정보를 가진 값을 만들기 위해 sin, cos 함수를 사용(pos은 입력 문장에서의 임베딩 벡터의 위치를 나타냄. (pos, i)의 i는 임베딩 벡터 내의 차원의 인덱스를 의미)
- 임베딩 벡터 내의 차원의 인덱스가 짝수인 경우에는 sim 함수의 값을 사용하고, 홀수인 경우에는 cos 함수의 값을 사용하는 구조
- 각 임베딩 벡터에 포지셔널 인코딩의 값을 더하면 같은 단어라고 하더라도 문장 내의 위치에 따라 트랜스포머의 입력으로 들어가는 임베딩 벡터의 값이 달라짐

### 1-3. 트랜스포머 셀프 어텐션
- 어텐션을 스스로에서 수행한다는 의미
- 셀프 어텐션은 하나의 문장 내에서 단어간의 관계를 파악하기 위해 사용하는 어텐션 매커니즘으로 Query, Key, Value가 동일
- Q,K,V 벡터를 얻었다면 기존의 어텐션 메커니즘과 동일하게 작동 -> 각 Q 벡터는 모든 K벡터에 대해서 어텐션 스코어를 구하고, 어텐션 분포를 구한 뒤에 모든 V벡터를 가중합해서 어텐션 값 또는 켄텍스트 벡터를 구할 수 있고 이를 Q벡터에 대해 반복
- 각 단어에 대한 Q,K,V 벡터를 구하고 스케일드 닷 프로덕트 어텐션을 수행했던 과정들은 벡터 연산이 아니라 행렬 연산을 사용하면 일괄 연산이 가능

### 1-4. 멀티헤드 어텐션
- 어텐션에는 d_model의 차원을 가진 단어 벡터를 num_heads 로 나눈 차원으로 어텐션을 수행
- 트랜스포머 연구진은 한 번의 어텐션을 하는 것 보다 여러 번의 어텐션을 병렬로 사용하는 것이 더 효과적이라고 판단
- d_model의 단어 임베딩을 차원 축소하는데 사용되는 가중치 행렬의 값은 8개의 어텐션 헤드마다 전부 다름
- 병렬 어텐션을 모두 수행하면 모두 연결된 어텐션 헤드 행렬의 크기가 (seq_len,d_model)

### 1-5. Position-wise FFNN(Feed Forward Neural Network)
- 우리가 알고있는 일반적인 deep neural network의 feed forward 신경망
- 각각의 학습 노드가 서로 완전하게 연결된 Fully-connected NN라고 해석할수 있음

### 1-6. 잔차 연결과 레이어 정규화
- sublayer의 출력에 입력 값을 더해주는 기법
- 입력과 출력은 FFNN을 지나기 때문에 동일한 차원을 가지므로 덧셈이 가능
- 잔차 연결을 거친 결과는 Layer Normalization 과정을 거침
- 수식으로 구현된 인코더는 총 num_layers 만큼을 순차적으로 한 후에 마지막 층의 인코더의 출력을 디코더에서 전달하면서 디코더 연산이 시작

### 1-7. 디코더 (Decoder)
- 디코더도 인코더와 동일하게 임베딩 층과 포지셔널 인코딩을 거친 후에 문장 행렬이 입력
- 학습시 Teacher Forcing 기법을 사용하여 학습되므로 학습 과정에서 디코더는 정답 문장에 해당하는 문장행렬을 한 번에 입력
- seq2seq는 정답을 하나씩 넣어주지만, 트랜스포머는 문장을 통째로 넣어주기 떄문에, 현 시점의 정답이 아니라 이후에 나올 정답 단어들까지 참조
- seq2seq는 정답을 하나씩 넣어주지만, 트랜스포머는 문장을 통째로 넣어주기 때문에, 현 시점의 정답이 아니라 이후에 나올 정답 단어들까지 참조 -> Look-ahead mask를 사용